In [ ]:
import csv
import time as t
import winsound

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def topic_page(url):
    
    t.sleep(3)
    try:
        response = requests.get(url)
    except:
        t.sleep(5)
        try:
            response = requests.get(url)
        except:
            return
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for article in soup.find_all('article', {'class':'story'}):
        article_ = {}
        
        title = article.find('h3', {'class':'story-title'}).get_text()
        short_text = article.find('p').get_text()
        time = article.find('time', {'class':'article-time'}).get_text()        
        # url
        d_url = article.find('div', {'class':'story-content'}).find('a').get('href')
        d_url = 'https://www.reuters.com' + d_url
        
        print (d_url)

        #detail
        detail = requests.get(d_url)
        if (detail.status_code == 200):
            detail = BeautifulSoup(detail.content,'html.parser')
        else:
            print('sleep')
            t.sleep(5)
            detail = requests.get(d_url)
            detail = BeautifulSoup(detail.content,'html.parser')
            if(detail.status_code != 200):
                continue
        
        long_text = detail.find('div',{'class':'ArticleBody_body_2ECha'}).get_text()
        
        # some little cleanings
        for char in ['\t','\n','  ']:
            title = title.replace(char,'')
            short_text = short_text.replace(char,'')
            time = time.replace(char,'')
            long_text = long_text.replace(char,'')
        
        article_['url'] = d_url
        article_['title'] = title
        article_['tag'] = ''
        article_['time'] = time
        article_['short_text'] = short_text
        article_['long_text'] = long_text
        
        with open('deal_reuter.csv', 'a',encoding = 'utf-8') as f:
            keys = ['url','title','tag','time','short_text','long_text']
            row = ','.join('"%s"' % article_[key].replace('"', '') for key in keys)
            f.write(row)
            f.write('\n')


In [ ]:
topic = 'innovationNews'
url = 'https://www.reuters.com/news/archive/'+topic+'?view=page&page='
for x in range(1,51):
    print (x)
    topic_page(url + str(x))
    

# now the search page

In [ ]:
def search_page(url):
    t.sleep(3)
    try:
        response = requests.get(url)
        zxc
    except:
        t.sleep(5)
        try:
            response = requests.get(url)
        except:
            return
    
    # the page return a javascript file contain the data, not json
    soup = BeautifulSoup(response.content, 'html.parser')
    soup = str(soup)
    soup = soup.replace('  ','')
    soup = soup.replace('\n','')
    soup = soup.replace('\t','')
    soup = soup.replace('\r','')
    soup = soup.partition('news: [')[-1].rpartition(']')[0] # get string between "news: [" and "]"
    soup = soup.split('},')
    
    count = 1
    for s in soup:
        try:
            article_ = {}
            
            href = s.partition('href: "')[-1].rpartition('",blu')[0] # "blu" is after the href attr we want

            detail_link = 'https://www.reuters.com' + href

            r = requests.get(detail_link)
            if (r.status_code==200):
                article = BeautifulSoup(r.content, 'html.parser')
            else:
                print('sleep')
                t.sleep(5)
                r = requests.get(detail_link)
                article = BeautifulSoup(r.content, 'html.parser')
                if(r.status_code!=200):
                    continue


            print (count, detail_link)
            count = count + 1

            article_['url'] = detail_link
            article_['title'] = article.find('h1',{'class':'ArticleHeader_headline_2zdFM'}).get_text()
            article_['tag'] = ''
            article_['time'] = article.find('div',{'class':'ArticleHeader_date_V9eGk'}).get_text()
            article_['short_text'] = ''
            article_['long_text'] = article.find('div',{'class':'ArticleBody_body_2ECha'}).get_text()

            keys = ['url','title','tag','time','short_text','long_text']

            with open('new_ceo_reuter.csv', 'a',encoding = 'utf8') as f:
                keys = ['url','title','tag','time','short_text','long_text']
                row = ','.join('"%s"' % article_[key].replace('"', '') for key in keys)
                f.write(row)
                f.write('\n')
        except:
            pass

In [ ]:
for x in range(1,2):
    search_string = 'new+ceo'
    url = 'https://www.reuters.com/assets/searchArticleLoadMoreJson?blob='+search_string+'&bigOrSmall=big&articleWithBlog=true&sortBy=&dateRange=&numResultsToShow=10&pn='+str(x)+'&callback=addMoreNewsResults'
    print (x)
    search_page(url)